In [1]:
!pip install --upgrade mxnet

     |████████████████████████████████| 68.7 MB 118.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install autogluon

     |████████████████████████████████| 515 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 16.5 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 31.6 MB/s eta 0:00:01
     |████████████████████████████████| 10.4 MB 24.7 MB/s eta 0:00:01    |████████████████▍               | 5.3 MB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 64.8 MB 66 kB/s s eta 0:00:01
     |████████████████████████████████| 752 kB 109.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 83.0 MB/s eta 0:00:01
     |████████████████████████████████| 882 kB 78.5 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 16.3 MB/s eta 0:00:01
  Created wheel for openml: filename=openml-0.10.2-py3-none-any.whl size=190318 sha256=3c0e3ec8824529f344e907df7e1b337ada73fb9a721ce3475ba3d70cecb4e401
  Stored in directory: /home/ec2-user/.cache/pip/wheels/45/83/27/a45cc845ff2497e34bfc17403e

In [3]:
import autogluon as ag

In [4]:
from autogluon import TabularPrediction as task

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import datetime
import gc
from sklearn.metrics import mean_squared_error

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 

In [6]:
# Download and unzip the data
!aws s3 cp s3://check-ride-data-explore/input/ashrae-energy-prediction/building_metadata.csv building_metadata.csv
    

download: s3://check-ride-data-explore/input/ashrae-energy-prediction/building_metadata.csv to ./building_metadata.csv


In [7]:
# Download and unzip the data
!aws s3 cp s3://check-ride-data-explore/input/ashrae-energy-prediction/test.csv test.csv
# Download and unzip the data
!aws s3 cp s3://check-ride-data-explore/input/ashrae-energy-prediction/train.csv train.csv
# Download and unzip the data
!aws s3 cp s3://check-ride-data-explore/input/ashrae-energy-prediction/weather_train.csv weather_train.csv
# Download and unzip the data
!aws s3 cp s3://check-ride-data-explore/input/ashrae-energy-prediction/weather_test.csv weather_test.csv

download: s3://check-ride-data-explore/input/ashrae-energy-prediction/test.csv to ./test.csv
download: s3://check-ride-data-explore/input/ashrae-energy-prediction/train.csv to ./train.csv
download: s3://check-ride-data-explore/input/ashrae-energy-prediction/weather_train.csv to ./weather_train.csv
download: s3://check-ride-data-explore/input/ashrae-energy-prediction/weather_test.csv to ./weather_test.csv


In [8]:
train_df = pd.read_csv('./train.csv')
building_df = pd.read_csv('./building_metadata.csv')
weather_df = pd.read_csv('./weather_train.csv')
weather_test_df = pd.read_csv('./weather_test.csv')

In [9]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype
def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
 #           df[col] = df[col].astype("category")
             pass

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [10]:
train_df = reduce_mem_usage(train_df,use_float16=True)
building_df = reduce_mem_usage(building_df,use_float16= True)

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.1%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.03 MB
Decreased by 60.3%


In [11]:
class WeatherTranformer(BaseEstimator, TransformerMixin):
    
    def __init__(self,weather_fillna=True):
        self._weather_fillna = weather_fillna
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        if self._weather_fillna:
            # Find Missing Dates
            time_format = "%Y-%m-%d %H:%M:%S" 
            start_date = datetime.datetime.strptime(X['timestamp'].min(),time_format)
            end_date = datetime.datetime.strptime(X['timestamp'].max(),time_format)
            total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
            hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]
            
            missing_hours = []
            for site_id in X.site_id.unique():
                site_hours = np.array(X[X['site_id'] == site_id]['timestamp'])
                new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
                new_rows['site_id'] = site_id
                X = pd.concat([X,new_rows])
                X = X.reset_index(drop=True) 
                
            # Add new Features
            X["datetime"] = pd.to_datetime(X["timestamp"])
            X["day"] = X["datetime"].dt.day
            X["week"] = X["datetime"].dt.week
            X["month"] = X["datetime"].dt.month    
            
            # Reset Index for Fast Update
            X = X.set_index(['site_id','day','month'])
            
            air_temperature_filler = pd.DataFrame(X.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
            X.update(air_temperature_filler,overwrite=False)
            
            # Step 1
            cloud_coverage_filler = X.groupby(['site_id','day','month'])['cloud_coverage'].mean()
            # Step 2
            cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])
            
            X.update(cloud_coverage_filler,overwrite=False)
            
            due_temperature_filler = pd.DataFrame(X.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
            X.update(due_temperature_filler,overwrite=False)
            
            # Step 1
            sea_level_filler = X.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
            # Step 2
            sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])
            X.update(sea_level_filler,overwrite=False)
            
            wind_direction_filler =  pd.DataFrame(X.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
            X.update(wind_direction_filler,overwrite=False)
            
            wind_speed_filler =  pd.DataFrame(X.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
            X.update(wind_speed_filler,overwrite=False)
            
            # Step 1
            precip_depth_filler = X.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
            # Step 2
            precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])
            
            X.update(precip_depth_filler,overwrite=False)
            
            X = X.reset_index()
            X = X.drop(['datetime','day','week','month'],axis=1)
    
        return X

In [12]:
weather_train_df = WeatherTranformer(True).fit_transform(weather_df)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [13]:
## Smoothness filter for Time series data

from scipy.signal import savgol_filter as sg

class SGFilterTranformer(BaseEstimator, TransformerMixin):
    
    def __init__(self,SGFilter=True):
        self._SGFilter = SGFilter
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        w = 11
        p = 2
        if self._SGFilter:
            for si in X.site_id.unique():
                index = X.site_id == si
                X.loc[index, 'air_smooth'] = sg(X[index].air_temperature, w, p)
                X.loc[index, 'dew_smooth'] = sg(X[index].dew_temperature, w, p)
                X.loc[index, 'air_diff'] = sg(X[index].air_temperature, w, p, 1)
                X.loc[index, 'dew_diff'] = sg(X[index].dew_temperature, w, p, 1)
                X.loc[index, 'air_diff2'] = sg(X[index].air_temperature, w, p, 2)
                X.loc[index, 'dew_diff2'] = sg(X[index].dew_temperature, w, p, 2)
        return X

In [14]:
weather_train_df = SGFilterTranformer(True).fit_transform(weather_train_df)
weather_train_df.head()

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,timestamp,wind_direction,wind_speed,air_smooth,dew_smooth,air_diff,dew_diff,air_diff2,dew_diff2
0,0,25.0,6.000000,20.0,-0.173913,1019.7,2016-01-01 00:00:00,0.0,0.0,25.163636,20.588112,-1.559394,-0.048438,0.242424,0.009324
1,0,24.4,4.285714,21.1,-1.000000,1020.2,2016-01-01 01:00:00,70.0,1.5,23.725455,20.544336,-1.316970,-0.039114,0.242424,0.009324
2,0,22.8,2.000000,21.1,0.000000,1020.2,2016-01-01 02:00:00,0.0,0.0,22.529697,20.509883,-1.074545,-0.029790,0.242424,0.009324
3,0,21.1,2.000000,20.6,0.000000,1020.1,2016-01-01 03:00:00,0.0,0.0,21.576364,20.484755,-0.832121,-0.020466,0.242424,0.009324
4,0,20.0,2.000000,20.0,-1.000000,1020.0,2016-01-01 04:00:00,250.0,2.6,20.865455,20.468951,-0.589697,-0.011142,0.242424,0.009324


In [15]:
## Lag feature

class RollwinTranformer(BaseEstimator, TransformerMixin):
    
    def __init__(self,Rollwin=True,window=3):
        self._Rollwin = Rollwin
        self._window = window
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        if self._Rollwin:
            group_df = X.groupby('site_id')
            cols = ['air_temperature', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
            rolled = group_df[cols].rolling(window=self._window, min_periods=0)
            lag_mean = rolled.mean().reset_index().astype(np.float16)
            lag_max = rolled.max().reset_index().astype(np.float16)
            lag_min = rolled.min().reset_index().astype(np.float16)
            lag_std = rolled.std().reset_index().astype(np.float16)
            for col in cols:
                X[f'{col}_mean_lag{self._window}'] = lag_mean[col]
                X[f'{col}_max_lag{self._window}'] = lag_max[col]
                X[f'{col}_min_lag{self._window}'] = lag_min[col]
                X[f'{col}_std_lag{self._window}'] = lag_std[col]
        return X

In [16]:
#weather_train_df = RollwinTranformer(True,48).fit_transform(weather_train_df)
weather_train_df = RollwinTranformer(True,24).fit_transform(weather_train_df)   
weather_train_df.iloc[:,10:].head()

,dew_smooth,air_diff,dew_diff,air_diff2,dew_diff2,air_temperature_mean_lag24,air_temperature_max_lag24,air_temperature_min_lag24,air_temperature_std_lag24,dew_temperature_mean_lag24,...,sea_level_pressure_min_lag24,sea_level_pressure_std_lag24,wind_direction_mean_lag24,wind_direction_max_lag24,wind_direction_min_lag24,wind_direction_std_lag24,wind_speed_mean_lag24,wind_speed_max_lag24,wind_speed_min_lag24,wind_speed_std_lag24
0,20.588112,-1.559394,-0.048438,0.242424,0.009324,25.000000,25.0,25.000000,NaN,20.000000,...,1019.5,NaN,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.0,NaN
1,20.544336,-1.316970,-0.039114,0.242424,0.009324,24.703125,25.0,24.406250,0.424316,20.546875,...,1019.5,0.353516,35.000000,70.0,0.0,49.50000,0.750000,1.500000,0.0,1.060547
2,20.509883,-1.074545,-0.029790,0.242424,0.009324,24.062500,25.0,22.796875,1.137695,20.734375,...,1019.5,0.288574,23.328125,70.0,0.0,40.40625,0.500000,1.500000,0.0,0.866211
3,20.484755,-0.832121,-0.020466,0.242424,0.009324,23.328125,25.0,21.093750,1.750000,20.703125,...,1019.5,0.238037,17.500000,70.0,0.0,35.00000,0.375000,1.500000,0.0,0.750000
4,20.468951,-0.589697,-0.011142,0.242424,0.009324,22.656250,25.0,20.000000,2.123047,20.562500,...,1019.5,0.207397,64.000000,250.0,0.0,108.31250,0.819824,2.599609,0.0,1.188477


In [17]:
weather_train_df = reduce_mem_usage(weather_train_df,use_float16=True)
weather_train_df.columns

Memory usage of dataframe is 22.52 MB
Memory usage after optimization is: 11.12 MB
Decreased by 50.6%


Index(['site_id', 'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'timestamp',
       'wind_direction', 'wind_speed', 'air_smooth', 'dew_smooth', 'air_diff',
       'dew_diff', 'air_diff2', 'dew_diff2', 'air_temperature_mean_lag24',
       'air_temperature_max_lag24', 'air_temperature_min_lag24',
       'air_temperature_std_lag24', 'dew_temperature_mean_lag24',
       'dew_temperature_max_lag24', 'dew_temperature_min_lag24',
       'dew_temperature_std_lag24', 'precip_depth_1_hr_mean_lag24',
       'precip_depth_1_hr_max_lag24', 'precip_depth_1_hr_min_lag24',
       'precip_depth_1_hr_std_lag24', 'sea_level_pressure_mean_lag24',
       'sea_level_pressure_max_lag24', 'sea_level_pressure_min_lag24',
       'sea_level_pressure_std_lag24', 'wind_direction_mean_lag24',
       'wind_direction_max_lag24', 'wind_direction_min_lag24',
       'wind_direction_std_lag24', 'wind_speed_mean_lag24',
       'wind_speed_max_lag24', 'wind_speed_

In [18]:
train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_train_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_train_df
gc.collect()
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,sea_level_pressure_min_lag24,sea_level_pressure_std_lag24,wind_direction_mean_lag24,wind_direction_max_lag24,wind_direction_min_lag24,wind_direction_std_lag24,wind_speed_mean_lag24,wind_speed_max_lag24,wind_speed_min_lag24,wind_speed_std_lag24
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,...,1019.5,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,...,1019.5,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,...,1019.5,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,...,1019.5,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,...,1019.5,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN


In [19]:
#Custom transformer we wrote to engineer features ( bathrooms per bedroom and/or how old the house is in 2019  ) 
#passed as boolen arguements to its constructor
class NumericalTransformer(BaseEstimator, TransformerMixin):
    #Class Constructor
    def __init__( self, outlier_1099=True,site0_corr=True, zone_corr=True, floor_cal=True, square_log=True,time_process=True, years_old=True):
        self._outlier_1099 = outlier_1099
        self._site0_corr = site0_corr
        self._zone_corr = zone_corr
        self._floor_cal = floor_cal
        self._square_log = square_log
        self._time_process = time_process
        self._years_old = years_old
        
    #Return self, nothing else to do here
    def fit( self, X, y = None ):
        return self 
    
    #Custom transform method we wrote that creates aformentioned features and drops redundant ones 
    def transform(self, X, y = None):
        #Check if needed 
        if self._outlier_1099:
            # Remove outliers
            X = X [ ~(X['building_id'] == 1099) ]
        if self._site0_corr:    
            #X = X.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')
            X = X[~((X['building_id']<=104) & (X['meter']==0) & (X['timestamp'] <= "2016-05-20"))]
            
        X.loc[:,'timestamp'] =  pd.to_datetime(X['timestamp'])
        
        if self._zone_corr:
            zone_dict={0:4,1:0,2:7,3:4,4:7,5:0,6:4,7:4,8:4,9:5,10:7,11:4,12:0,13:5,14:4,15:4} 
            for sid, zone in zone_dict.items():
                sids = X.site_id == sid
                
                X.loc[sids,'timestamp'] = X.loc[sids,'timestamp']-pd.offsets.Hour(int(zone))
        
        if self._floor_cal:
            X.loc[:,'floor_area'] = X.square_feet / X.floor_count
            
        if self._square_log: 
            X.loc[:,'square_feet'] =  np.log1p(X['square_feet'])
        
        if self._time_process:
            # Add more features
            X.loc[:,"timestamp"] = pd.to_datetime(X.loc[:,"timestamp"],format="%Y-%m-%d %H:%M:%S")
            X.loc[:,"hour"] = X["timestamp"].dt.hour
            X.loc[:,"weekend"] = X["timestamp"].dt.weekday
            X.loc[:,"week"] = X["timestamp"].dt.week
            X.loc[:,"month"] = X["timestamp"].dt.month    
            
        #Check if needed     
        if self._years_old:
            #create new column
            X.loc[:,'yr_built'] =  2020 - X['year_built']

         
        # Sort by timestamp
        X.sort_values("timestamp", inplace=True)
        X.reset_index(drop=True, inplace=True)
        X = X.drop(["floor_count",'year_built'], axis = 1 )
    
        #returns a numpy array
        return X

In [20]:
train_df = NumericalTransformer(True, True, False, True, True, True, True).fit_transform(train_df)
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,...,wind_speed_mean_lag24,wind_speed_max_lag24,wind_speed_min_lag24,wind_speed_std_lag24,floor_area,hour,weekend,week,month,yr_built
0,105,0,2016-01-01,23.303600,1,Education,10.832181,3.800781,0.0,2.400391,...,3.099609,3.099609,3.099609,NaN,10124.6,0,4,53,1,NaN
1,1124,0,2016-01-01,98.009003,13,Lodging/residential,11.940152,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN
2,1123,0,2016-01-01,27.444000,13,Office,11.440301,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN
3,1122,1,2016-01-01,0.000000,13,Office,11.342920,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN
4,1122,0,2016-01-01,70.580002,13,Office,11.342920,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN


In [21]:
class CategoricalTransformer( BaseEstimator, TransformerMixin ):
    #Class constructor method that takes in a list of values as its argument
    def __init__(self):
        pass
        
    #Return self nothing else to do here
    def fit( self, X, y = None):
        return self
    
    #Transformer method we wrote for this transformer 
    def transform(self, X , y = None ):
        ## Hardcoded city name for mapping
        city_dict = {0:'Jacksonville',1:'London',2:'Phoenix',3:'Philadelphia',4:'San Francisco',5:'Loughborough',6:'Philadelphia',7:'Montreal',8:'Jacksonville',9:'San Antonio',10:'Las Vegas',11:'Montreal',12:'Dublin',13:'Minneapolis',14:'Philadelphia',15:'Pittsburgh'}
        for sid, city in city_dict.items():
            sids = X['site_id'] == sid
            X.loc[sids, 'city'] = city
        
            # Encode Categorical Data
        le = LabelEncoder()
        X.loc[:,'primary_use'] = le.fit_transform(X["primary_use"])
   
        return X

In [22]:
train_df = CategoricalTransformer().fit_transform(train_df)

In [23]:
class OutlierTransformer( BaseEstimator, TransformerMixin ):
    #Class constructor method that takes in a list of values as its argument
    def __init__(self, electric_outlier=True, chill_outlier=True, hot_outlier=True):
        self._electric_outlier = electric_outlier
        self._chill_outlier = chill_outlier
        self._hot_outlier = hot_outlier
        
    #Return self nothing else to do here
    def fit( self, X, y = None):
        return self
    
    #Transformer method we wrote for this transformer 
    def transform(self, X):
        funny_electric_bids = [803, 1088,  993, 799]
        funny_chill_bids = [778, 1088]
        funny_hot_bids = [1331,1021,794]
        
        if self._electric_outlier:
            for bid in funny_electric_bids:
                
                X = X [ ~((X['building_id'] == bid) & (X['meter'] == 0)) ]
        if self._chill_outlier:
            for bid in funny_chill_bids:
                
                X = X [ ~((X['building_id'] == bid) & (X['meter'] == 1)) ]
        if self._hot_outlier:
            for bid in funny_hot_bids:
                
                X = X [ ~((X['building_id'] == bid) & (X['meter'] == 3)) ]
        
        X = X.reset_index().drop('index',axis=1)
   
        return X

In [24]:
train_df = OutlierTransformer(True,True,True).fit_transform(train_df)
len(train_df)

19776147

In [25]:

train_df['meter_reading'] = np.log1p(train_df["meter_reading"])

train_df = train_df.drop(['city'], axis = 1)
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,...,wind_speed_mean_lag24,wind_speed_max_lag24,wind_speed_min_lag24,wind_speed_std_lag24,floor_area,hour,weekend,week,month,yr_built
0,105,0,2016-01-01,3.190624,1,0,10.832181,3.800781,0.0,2.400391,...,3.099609,3.099609,3.099609,NaN,10124.6,0,4,53,1,NaN
1,1124,0,2016-01-01,4.595211,13,4,11.940152,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN
2,1123,0,2016-01-01,3.347937,13,6,11.440301,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN
3,1122,1,2016-01-01,0.000000,13,6,11.342920,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN
4,1122,0,2016-01-01,4.270816,13,6,11.342920,-8.296875,8.0,-12.203125,...,3.166016,9.000000,0.000000,2.179688,NaN,0,4,53,1,NaN


In [26]:
train_df.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'air_smooth', 'dew_smooth', 'air_diff',
       'dew_diff', 'air_diff2', 'dew_diff2', 'air_temperature_mean_lag24',
       'air_temperature_max_lag24', 'air_temperature_min_lag24',
       'air_temperature_std_lag24', 'dew_temperature_mean_lag24',
       'dew_temperature_max_lag24', 'dew_temperature_min_lag24',
       'dew_temperature_std_lag24', 'precip_depth_1_hr_mean_lag24',
       'precip_depth_1_hr_max_lag24', 'precip_depth_1_hr_min_lag24',
       'precip_depth_1_hr_std_lag24', 'sea_level_pressure_mean_lag24',
       'sea_level_pressure_max_lag24', 'sea_level_pressure_min_lag24',
       'sea_level_pressure_std_lag24', 'wind_direction_mean_lag24',
       'wind_direction_max_lag24', 'wind_direction_min_lag24',
       'wind_direction_st

In [27]:
label_column = 'meter_reading'
print("Summary of class variable: \n", train_df[label_column].describe())

Summary of class variable: 
 count    1.977615e+07
mean     4.268331e+00
std      2.031802e+00
min      0.000000e+00
25%      3.052429e+00
50%      4.413101e+00
75%      5.607455e+00
max      1.261142e+01
Name: meter_reading, dtype: float64


In [28]:
dir = 'agModels-simplefit-10percent' # specify folder where to store trained models

### Get only 10 percent of data for training

In [29]:

target = train_df["meter_reading"]
features = train_df.drop(['meter_reading'], axis = 1)
kf = KFold(n_splits=10,shuffle=True)
train_index,test_index = next(kf.split(features))
train_features = features.loc[test_index]
train_target = target.loc[test_index]


In [30]:
len(train_features[train_features['meter']==0])

1166042

In [31]:
train_target.head()

4     4.270816
13    7.334493
19    7.597492
20    3.068416
38    3.091043
Name: meter_reading, dtype: float32

In [32]:
train_data = pd.merge(train_features, train_target, left_index=True, right_index=True)
train_data.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,...,wind_speed_max_lag24,wind_speed_min_lag24,wind_speed_std_lag24,floor_area,hour,weekend,week,month,yr_built,meter_reading
4,1122,0,2016-01-01,13,6,11.342920,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,4.270816
13,1117,2,2016-01-01,13,6,11.621107,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,7.334493
19,1112,2,2016-01-01,13,6,12.157260,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,7.597492
20,1111,2,2016-01-01,13,6,11.124907,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,3.068416
38,1141,0,2016-01-01,13,8,9.951658,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,3.091043


In [33]:
train_data.reset_index(inplace=True)

In [34]:
len(train_data)

1977615

In [35]:
train_data.drop('index',axis=1,inplace=True)
train_data.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,...,wind_speed_max_lag24,wind_speed_min_lag24,wind_speed_std_lag24,floor_area,hour,weekend,week,month,yr_built,meter_reading
0,1122,0,2016-01-01,13,6,11.342920,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,4.270816
1,1117,2,2016-01-01,13,6,11.621107,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,7.334493
2,1112,2,2016-01-01,13,6,12.157260,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,7.597492
3,1111,2,2016-01-01,13,6,11.124907,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,3.068416
4,1141,0,2016-01-01,13,8,9.951658,-8.296875,8.0,-12.203125,-0.217407,...,9.0,0.0,2.179688,NaN,0,4,53,1,NaN,3.091043


In [ ]:
predictor = task.fit(train_data=train_data, label=label_column, auto_stack=True,output_directory=dir,feature_prune=True,excluded_model_types=['KNN', 'NN'])
results = predictor.fit_summary()

Beginning AutoGluon training ...
AutoGluon will save models to agModels-simplefit-10percent/
AutoGluon Version:  0.0.12
Train Data Rows:    1977615
Train Data Columns: 50
Preprocessing data ...
Here are the first 10 unique label values in your data:  [4.270815849304199, 7.3344926834106445, 7.59749174118042, 3.068415641784668, 3.0910425186157227, 7.2087554931640625, 4.836281776428223, 2.9042747020721436, 5.262275695800781, 6.844282627105713]
AutoGluon infers your prediction problem is: regression  (because dtype of label-column == float and many unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Feature Generator processed 1977615 data points with 49 features
Original Features (raw dtypes):
	int16 features: 1
	int8 features: 2
	datetime64[ns] features: 1
	int64 features: 5
	float64 features: 2
	float16 features: 37
	float32 features: 1
Original Features

In [34]:
predictor.feature_importance(train_data)

Computing raw permutation importance for 50 features on weighted_ensemble_k0_l1 ...
	4894.75s	= Expected runtime
	1698.42s	= Actual runtime


square_feet                      1.590385
meter                            1.464159
building_id                      0.730670
site_id                          0.539470
primary_use                      0.477466
yr_built                         0.327239
air_smooth                       0.184938
floor_area                       0.157202
hour                             0.127135
index                            0.104978
air_temperature                  0.098691
timestamp                        0.087770
month                            0.069046
week                             0.065445
air_temperature_mean_lag24       0.063114
weekend                          0.058558
dew_smooth                       0.054178
air_temperature_max_lag24        0.038689
air_temperature_min_lag24        0.032993
dew_temperature                  0.032328
precip_depth_1_hr_max_lag24      0.021860
dew_temperature_mean_lag24       0.019880
dew_temperature_min_lag24        0.018747
cloud_coverage                   0

In [29]:
!aws s3 cp ./agModels-simplefit-10percent/ s3://check-ride-data-explore/agModels-10percent --recursive

upload: agModels-simplefit-10percent/learner.pkl to s3://check-ride-data-explore/agModels-10percent/learner.pkl
upload: agModels-simplefit-10percent/SummaryOfModels.html to s3://check-ride-data-explore/agModels-10percent/SummaryOfModels.html
upload: agModels-simplefit-10percent/models/CatboostRegressor/model.pkl to s3://check-ride-data-explore/agModels-10percent/models/CatboostRegressor/model.pkl
upload: agModels-simplefit-10percent/models/LightGBMRegressor/model.pkl to s3://check-ride-data-explore/agModels-10percent/models/LightGBMRegressor/model.pkl
upload: agModels-simplefit-10percent/models/NeuralNetRegressor/temp_net.params to s3://check-ride-data-explore/agModels-10percent/models/NeuralNetRegressor/temp_net.params
upload: agModels-simplefit-10percent/models/LightGBMRegressorCustom/model.pkl to s3://check-ride-data-explore/agModels-10percent/models/LightGBMRegressorCustom/model.pkl
upload: agModels-simplefit-10percent/models/trainer.pkl to s3://check-ride-data-explore/agModels-10p